In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns 
from networkx.algorithms import tree

#  Algoritmos de búsqueda

La matriz de acceso de un grafo G, dirigido o no dirigido, puede obtenerse a raíz de la aplicación reiterada de dos métodos diferentes:

- BFS: Breadth First Search, Búsqueda en extensión o anchura
- DFS: Depth First Search, Búsqueda en profundidad

Ambos algoritmos nos permiten _visitar_ todos los vértices y arcos del grafo exactamente uan vez y en un orden específico. 


## Algoritmo BFS

Este algoritmo hace la búsqueda de nodos por niveles, tal y como puede verse en la imagen

<img src="img/bfs.png">

Vamos a empezar implementando el algoritmo nosotros mismos. Tengamos el siguiente grafo, definido como un diccionario, en el que los valores de las claves son sets que contienen los vértices con los que cada vértice está enlazado.
Recordemos que un set es un conjunto no ordenador de elementos únicos

In [ ]:
graph = {'A': set(['B', 'C']),
         'B': set(['A', 'E', 'D']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}

Con la siguiente implementación obtenemos el conjunto de vértices accesibles desde el nodo inicial

In [ ]:
def bfs(graph, start):
    visited, queue = set(), [start] #inicializamos el conjunto de nodos visitados a un set vacío
    while queue:
        vertex = queue.pop(0) #pop elimina el ultimo elemento de una lista. Con índice 0 elimina el primero
        if vertex not in visited:
            visited.add(vertex) #si aún no hemos pasado por el vértice, se añade a la lista de visitados
            queue.extend(graph[vertex] - visited) #eliminamos del set de nodos enlazados al nodo de estudio, aquellos por los que ya hemos pasado
    return visited

In [ ]:
bfs(graph, 'A')

Podemos también obtener el camino seguido para llegar de un nodo a otro. Para ello en este caso usamos un generador. Aunque parezca una función, podemos ver que no contiene al argumento _return_. 

Cuando hablamos de generador nos referimos a "funcion-generador".
Es una funcion normal, que contiene un "yield".
Este concepto se utiliza para crear valores __iterables__, que gracias al "yield" nos van a devolver los valores de uno en uno, suspendiendo el proceso hasta que vuelve a ser llamado (generalmente, en la siguiente iteración del bucle).

In [ ]:
def bfs_paths(graph, start, goal):
    queue = [(start, [start])] #generamos una lista de tuplas, con el valor del nodo y el camino
    while queue:
        (vertex, path) = queue.pop(0)
        for elemento in graph[vertex] - set(path): 
            if elemento == goal:
                yield path + [elemento] #si el nodo es el nodo final, para la ejecucción hasta que vuelva a recibir datos
            else:
                queue.append((elemento, path + [elemento]))

Observemos que el objeto resultante al aplicar el generador no devuelve nada, pues es un generador

In [ ]:
bfs_paths(graph, 'A', 'F')

Para ello deberemos devolverlo en forma de un iterable, en este caso, una lista.

In [ ]:
list(bfs_paths(graph, 'A', 'F'))

Existen dos caminos posibles del nodo 'A' a 'F'.

## Algoritmo DFS

En este caso la búsqueda del algoritmo la hace en profundidad. Es decir, recorre el grafo desde el nodo de salida hasta los nodos extremos u hojas por cada una de las ramas. En este caso los niveles de cada nodo no son importantes.

<img src="img/dfs.png">

Vmaos a implementar, al igual que hicimos con BFS, nosotros el algoritmo. Pero esta vez lo haremos partiendo de un grafo definido con listas en lugar de sets

In [ ]:
graph1 = {
    'A' : ['B','S'],
    'B' : ['A'],
    'C' : ['D','E','F','S'],
    'D' : ['C'],
    'E' : ['C','H'],
    'F' : ['C','G'],
    'G' : ['F','S'],
    'H' : ['E','G'],
    'S' : ['A','C','G']
}

In [ ]:
def dfs(graph, node, visited):
    if node not in visited:
        visited.append(node)
        for n in graph[node]:
            dfs(graph,n, visited)
    return visited

In [ ]:
visited = dfs(graph1,'A', [])
print(visited)

Si quisiéramos implementarlo para un grafo con sets, tal y como hicimos con BFS, nótese que la función es igual con la excepción de que la lista de vértices en este caso elimina el último elemento de la lista, en lugar dle primero

In [ ]:
graph2 = {
    'A' : set(['B','S']),
    'B' : set(['A']),
    'C' : set(['D','E','F','S']),
    'D' : set(['C']),
    'E' : set(['C','H']),
    'F' : set(['C','G']),
    'G' : set(['F','S']),
    'H' : set(['E','G']),
    'S' : set(['A','C','G'])
}

In [ ]:
def dfs(graph, start):
    visited, stack = set(), [start] #inicializamos el conjunto de nodos visitados a un set vacío
    while stack:
        vertex = stack.pop() #elimina el último elemento de la lista
        if vertex not in visited:
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
    return visited

In [ ]:
dfs(graph2, 'A')

Hay que recordar que los sets son conjuntos no ordenados, por lo que el resultado al trabajar con sets nos devuelve el conjunto de vértices por los que pasa, pero no en qué orden. Al realizarlo con listas, sin embargo, si que obtenemos el orden al recorrer el grafo.

## Algoritmos de búsqueda con NetworkX

Vamos a definir el mismo grafo que teníamos para el ejemplo de dfs pero con networkx.

In [ ]:
G = nx.Graph()
G.add_edge("A","B")
G.add_edge("A","S")
G.add_edge("C","D")
G.add_edge("C","E")
G.add_edge("C","F")
G.add_edge("C","S")
G.add_edge("E","H")
G.add_edge("F","G")
G.add_edge("H","G")
G.add_edge("S","G")

In [ ]:
nx.draw(G, with_labels = True)
plt.show()

A continuación vamos a aplicar los métodos que iteran sobre el grafo utilizando _BFS_ y _DFS_. Ambos nos devuelven los pares de vértices en el orden que los va recorriendo. Para aplicar _BFS_ hay que indicar el vértice de origen. Podemos ver como a partir del vértice _S_ el camino no es el mismo.

In [ ]:
list(nx.dfs_edges(G))

In [ ]:
list(nx.bfs_edges(G, 'A'))

Se puede visualizar el árbol que contruye al ir recorriendo los nodos para cada algoritmo de búsqueda.

In [ ]:
nx.draw(nx.dfs_tree(G), with_labels = True)

In [ ]:
nx.draw(nx.bfs_tree(G, 'A'), with_labels = True)

networkx nos devuelve los nodos predecesores y sucesores de cada nodo, según el aĺgoritmo de búsqueda usado.

In [ ]:
print("Aquí obtenemos los nodos predecesores para cada uno de ellos con DFS \n", nx.dfs_predecessors(G))
print("Aquí obtenemos los nodos sucesores para cada uno de ellos \n", nx.dfs_successors(G))

In [ ]:
print("Aquí obtenemos los nodos predecesores para cada uno de ellos con BFS \n", list(nx.bfs_predecessors(G, 'A')))
print("Aquí obtenemos los nodos sucesores para cada uno de ellos \n", list(nx.bfs_successors(G, 'A')))

Podemos ver como con _DFS_ el nodo predecesor de _G_ es _H_ mientras que con _BFS_ es _S_.